In [1]:
import requests
import string

from utils.get_data import get_user, get_project, get_all_projects, get_all_users
from utils.metric_utils import intersection_metric, iou_metric
from utils.similarity_utils import cosine_similarity
from utils.text_utils import embedd_representation, get_user_roles_desc, get_user_text_desc, get_user_skill_desc, get_project_text_desc
from utils.search_utils import (
    search_similar_projects,
    search_specialists_for_project,
    search_projects_for_specialist,
    search_users_for_user
)
from tqdm import tqdm
import pickle
import pandas as pd
import numpy as np
from scipy.spatial import distance
from typing import Dict, Any, List

In [2]:
test_parquet = pd.read_parquet('/Users/andrey.krotkikh/Documents/embedds.parquet')

In [6]:
get_all_users()

[{'_id': '636178e6a668d3e6cd82d39c',
  'userId': '635d8e808acf6112cb7d0cd0',
  'isEmailVerified': False,
  'isProfileCreated': False,
  'isFormCreated': False,
  'isPublished': False,
  'isDraft': False,
  'viewCounter': 0,
  'firstName': 'Михаил',
  'secondName': 'Марцинюк',
  'lastName': 'Сергеевич',
  'birthDate': '1998-01-22',
  'country': 'Россия',
  'city': 'Ростов-на-Дону',
  'citizenship': 'Российское',
  'gender': 'Мужской',
  'skillInformation': [{'category': 'Teamlead',
    'skills': ['Excel', 'CustDev', 'Управление командой'],
    'experience': 'senior',
    '_id': '636178e6a668d3e6cd82d39d'},
   {'category': 'Frontend',
    'skills': ['Angular 2+'],
    'experience': 'senior',
    '_id': '636178e6a668d3e6cd82d39e'}],
  'interestedTags': ['fintech',
   'foodtech',
   'biotech',
   'робототехника',
   'программное обеспечение',
   'продажи'],
  'experience': '100',
  'aboutDescription': 'Я директор IT-компании Dragons Code, мне 24 года, люблю робототехнику и искусственный ин

In [5]:
get_project('6362ddaae685eb7ddd6dd794')

{'_id': '6362ddaae685eb7ddd6dd794',
 'projectOwnerId': '636178e6a668d3e6cd82d39c',
 'type': 'innovator',
 'projectName': 'Space Bear',
 'projectDescription': 'Проект для конкурса "Лидеры Цифровой Трансформации 2022", разрабатывается командой Dragons. В проекте используется очень много ИИ, для рекомендации идеальных единомышленников, подбора проектов и поиска новых знакомств. \n\nВ проект требуется маркетолог для привлечения пользователей.',
 'isPublished': True,
 'projectTags': ['искусственный интеллект',
  'знакомства и общение',
  'стартапы'],
 'existTeam': [{'fullProfileId': '636178e6a668d3e6cd82d39c',
   'category': 'Teamlead',
   'skills': ['Управление командой'],
   '_id': '6362ddaae685eb7ddd6dd795'}],
 'lookingForTeam': [{'category': 'Frontend',
   'skills': ['Angular 2+', 'Node js', 'JavaScript', 'TypeScript'],
   '_id': '6362ddaae685eb7ddd6dd796'},
  {'category': 'Backend',
   'skills': ['Nest js', 'Node js', 'JavaScript', 'TypeScript'],
   '_id': '6362ddaae685eb7ddd6dd797'}],

In [4]:
get_all_projects()

[{'_id': '6362ddaae685eb7ddd6dd794',
  'projectOwnerId': '636178e6a668d3e6cd82d39c',
  'type': 'innovator',
  'projectName': 'Space Bear',
  'projectDescription': 'Проект для конкурса "Лидеры Цифровой Трансформации 2022", разрабатывается командой Dragons. В проекте используется очень много ИИ, для рекомендации идеальных единомышленников, подбора проектов и поиска новых знакомств. \n\nВ проект требуется маркетолог для привлечения пользователей.',
  'isPublished': True,
  'projectTags': ['искусственный интеллект',
   'знакомства и общение',
   'стартапы'],
  'existTeam': [{'fullProfileId': '636178e6a668d3e6cd82d39c',
    'category': 'Teamlead',
    'skills': ['Управление командой'],
    '_id': '6362ddaae685eb7ddd6dd795'}],
  'lookingForTeam': [{'category': 'Frontend',
    'skills': ['Angular 2+', 'Node js', 'JavaScript', 'TypeScript'],
    '_id': '6362ddaae685eb7ddd6dd796'},
   {'category': 'Backend',
    'skills': ['Nest js', 'Node js', 'JavaScript', 'TypeScript'],
    '_id': '6362ddaae

In [6]:
request_all_projects = requests.get("https://space-bear-api.herokuapp.com/api/projects/ml-get-all")
all_projects_dict = request_all_projects.json()

In [7]:
all_projects_dict

[{'_id': '6362ddaae685eb7ddd6dd794',
  'projectOwnerId': '636178e6a668d3e6cd82d39c',
  'type': 'innovator',
  'projectName': 'Space Bear',
  'projectDescription': 'Проект для конкурса "Лидеры Цифровой Трансформации 2022", разрабатывается командой Dragons. В проекте используется очень много ИИ, для рекомендации идеальных единомышленников, подбора проектов и поиска новых знакомств. \n\nВ проект требуется маркетолог для привлечения пользователей.',
  'isPublished': True,
  'projectTags': ['искусственный интеллект',
   'знакомства и общение',
   'стартапы'],
  'existTeam': [{'fullProfileId': '636178e6a668d3e6cd82d39c',
    'category': 'Teamlead',
    'skills': ['Управление командой'],
    '_id': '6362ddaae685eb7ddd6dd795'}],
  'lookingForTeam': [{'category': 'Frontend',
    'skills': ['Angular 2+', 'Node js', 'JavaScript', 'TypeScript'],
    '_id': '6362ddaae685eb7ddd6dd796'},
   {'category': 'Backend',
    'skills': ['Nest js', 'Node js', 'JavaScript', 'TypeScript'],
    '_id': '6362ddaae

In [5]:
get_project_text_desc("6362ddaae685eb7ddd6dd794")

'Проект для конкурса "Лидеры Цифровой Трансформации 2022", разрабатывается командой Dragons. В проекте используется очень много ИИ, для рекомендации идеальных единомышленников, подбора проектов и поиска новых знакомств. \n\nВ проект требуется маркетолог для привлечения пользователей.'

In [3]:
search_specialists_for_project("6362ddaae685eb7ddd6dd794", test_parquet)

/Users/andrey.krotkikh/spacebear/lib/python3.8/site-packages/numpy/core/fromnumeric.py:86: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)


{'636178e6a668d3e6cd82d39c': {'Frontend': 1,
  'Backend': 0,
  'semantic': 0.759360927367716},
 '6362832611ebb3047851dd45': {'Frontend': 0,
  'Backend': 0,
  'semantic': 0.734008252620697}}

In [4]:
search_similar_projects("6362ddaae685eb7ddd6dd794", test_parquet)

{'6362def7e685eb7ddd6dd7ce': 0.8435057428563579,
 '6362dfe8e685eb7ddd6dd7e7': 0.7547830859280128}

In [10]:
search_projects_for_specialist("636178e6a668d3e6cd82d39c", test_parquet)

/Users/andrey.krotkikh/spacebear/lib/python3.8/site-packages/numpy/core/fromnumeric.py:86: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)


{'6362ddaae685eb7ddd6dd794': {'Frontend': 1,
  'Backend': 0,
  'semantic': 0.759360927367716},
 '6362def7e685eb7ddd6dd7ce': {'Product Owner': 0,
  'Marketing': 0,
  'semantic': 0.8748704681199942},
 '6362dfe8e685eb7ddd6dd7e7': {'ML': 0,
  'Frontend': 1,
  'semantic': 0.7969735477427032}}

In [11]:
search_users_for_user("636178e6a668d3e6cd82d39c", test_parquet)

{'6362832611ebb3047851dd45': {'Interests': 1, 'Semantic': 0.7476101837173933}}

In [24]:
def innovation_score_project(project_id: str, input_df: pd.DataFrame) -> bool:
    desc = get_project(project_id)['projectDescription']

    innovative_desc = "инновации ии искусственный интеллект дополненная виртуальная реальность энергосбережение автоматизация роботы робототехника" \
                      "имплантация космос генерализация блокчейн криптовалюта нфт аи децентрализованный децентрализация метавселенная неинвазивный"

    metric = cosine_similarity(embedd_representation(desc, input_df), embedd_representation(innovative_desc, input_df))

    return metric > 0.75

In [25]:
innovation_score_project("6362ddaae685eb7ddd6dd794", test_parquet)

False